In [ ]:
from transformers import AutoImageProcessor, AutoModelForImageClassification
from PIL import Image
import cv2
import os
from collections import Counter

In [ ]:
processor = AutoImageProcessor.from_pretrained("dima806/facial_emotions_image_detection")
model = AutoModelForImageClassification.from_pretrained("dima806/facial_emotions_image_detection")a

In [ ]:
class_names = ['Sad', 'Disgust', 'Angry', 'Neutral', 'Fear', 'Surprise', 'Happy']

In [ ]:
# Function to predict emotion from an image
def predict_emotion(image):
    try:
        # Preprocess image for model input
        # image = image.convert('RGB')
        inputs = processor(images=image, return_tensors="pt")

        # Get predictions
        outputs = model(**inputs)
        logits = outputs.logits

        # Get top 3 emotions with their corresponding scores
        softmax_scores = logits.softmax(dim=1)
        top_3_scores, top_3_indices = softmax_scores.topk(3, dim=1)

        predictions = []
        for i in range(3):
            emotion = class_names[top_3_indices[0][i].item()]
            score = top_3_scores[0][i].item()
            predictions.append({"emotion": emotion, "score": score})

        return predictions
    except Exception as e:
        print(f"Error processing frame: {e}")
        return []

# Process a video file in 10-second chunks
def process_video(video_path, chunk_duration=4):
    # Check the fps of the video
    framespsec = int(cv2.VideoCapture(video_path).get(cv2.CAP_PROP_FPS))
    print(f"FPS: {framespsec}")
    try:
        # Open the video file
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            print(f"Error: Unable to open video {video_path}")
            return

        # Calculate frames per chunk
        frames_per_chunk = framespsec * chunk_duration
        frame_count = 0
        chunk_emotions = []

        # Read video frames
        while True:
            ret, frame = cap.read()
            if not ret:
                break

            # Process only every 30th frame
            if frame_count % 30 == 0:
                # Convert frame from BGR to RGB for PIL
                frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                frame_image = Image.fromarray(frame_rgb)

                # Predict emotions for the frame
                predictions = predict_emotion(frame_image)
                if predictions:
                    # Store the most likely emotion from top 3
                    chunk_emotions.append(predictions[0]['emotion'])

            # Check if we've processed the current chunk
            frame_count += 1
            if frame_count % frames_per_chunk == 0:
                # Analyze the chunk
                emotion_counter = Counter(chunk_emotions)
                top_2_emotions = emotion_counter.most_common(2)

                print(f"Chunk {frame_count // frames_per_chunk}:")
                for emotion, freq in top_2_emotions:
                    print(f"Emotion: {emotion} | Frequency: {freq}")

                # Reset for the next chunk
                chunk_emotions = []

        # Handle remaining frames in the last chunk
        if chunk_emotions:
            emotion_counter = Counter(chunk_emotions)
            top_2_emotions = emotion_counter.most_common(2)

            print(f"Chunk {frame_count // frames_per_chunk + 1}:")
            for emotion, freq in top_2_emotions:
                print(f"Emotion: {emotion} | Frequency: {freq}")

        cap.release()
    except Exception as e:
        print(f"Error processing video {video_path}: {e}")

In [ ]:
# Path to the video file
video_file = "beauty2.mov"  # Replace with your video file path

# Process the video
process_video(video_file)
